In [46]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy 
import pandas as pd
import pickle as pkl
import dataframe

In [47]:
'''df = pd.read_csv('data/tweets_ats.csv')
tweets = df['renderedContent'].tolist()'''

"df = pd.read_csv('data/tweets_ats.csv')\ntweets = df['renderedContent'].tolist()"

In [48]:
app_store = pkl.load(open("./data/app_store.pkl", "rb"))
app_store = pd.json_normalize(app_store)

play_store = pkl.load(open("./data/play_store.pkl", "rb"))
play_store = pd.json_normalize(play_store)
df = dataframe.GetDataframeStores(play_store,app_store)

c:\Users\yanni\OneDrive\FHNW\Challenges\MAKEathon\dataframe.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_app["content"] = df_app["title"] + ": " + df_app["review"]


In [49]:
df["lencontent"] = df["content"].apply(lambda x : len(x))
df = df[df["lencontent"] < 514]
df = df[df["language_prob"] == "en"]
tweets = df["content"].tolist()


In [50]:
# load model and tokenizer in English
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Positive', 'Neutral', 'Negative']

In [51]:
# Clustering the Sentences
positive = []
negative = []
neutral = []

count = 0

for tweet in tweets:
    count += 1
    print(count)
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
    
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    positive.append(scores[0])
    negative.append(scores[1])
    neutral.append(scores[2])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [52]:
# create a new dataframe with the sentiment scores
df_sentiment = pd.DataFrame({'positive': positive, 'negative': negative, 'neutral': neutral})
df_sentiment

,positive,negative,neutral
0,0.007042,0.042536,0.950422
1,0.769135,0.191246,0.039620
2,0.960920,0.035153,0.003927
3,0.454550,0.335977,0.209473
4,0.976221,0.019958,0.003821
...,...,...,...
522,0.909631,0.082988,0.007382
523,0.070144,0.751667,0.178189
524,0.003112,0.070366,0.926522
525,0.199186,0.759826,0.040988


In [56]:
df.reset_index(drop=True, inplace=True)
df

,rating,content,language_prob,lencontent
0,2,It was better before: In general I like the ap...,en,188
1,1,Poor limited functionality: The previous app w...,en,398
2,1,Horrible: The previous one was super. This is ...,en,141
3,1,Howto make a good app become the worst ever: A...,en,208
4,1,The worst bank and app of CH: I can’t tell eno...,en,212
...,...,...,...,...
522,1,...stll can't donload cause its not sutable fo...,en,74
523,5,Ich benutze die app mit htc desire hd. Alles t...,en,114
524,4,"[galaxS2] having an accountPOSTFINANCE, iz app...",en,115
525,5,Tut was es tun muss und dies tut es gut.,en,40


In [61]:
df_test = pd.concat([df[["content", 'rating', 'language_prob']], df_sentiment], axis=1)
df_test

,content,rating,language_prob,positive,negative,neutral
0,It was better before: In general I like the ap...,2,en,0.007042,0.042536,0.950422
1,Poor limited functionality: The previous app w...,1,en,0.769135,0.191246,0.039620
2,Horrible: The previous one was super. This is ...,1,en,0.960920,0.035153,0.003927
3,Howto make a good app become the worst ever: A...,1,en,0.454550,0.335977,0.209473
4,The worst bank and app of CH: I can’t tell eno...,1,en,0.976221,0.019958,0.003821
...,...,...,...,...,...,...
522,...stll can't donload cause its not sutable fo...,1,en,0.909631,0.082988,0.007382
523,Ich benutze die app mit htc desire hd. Alles t...,5,en,0.070144,0.751667,0.178189
524,"[galaxS2] having an accountPOSTFINANCE, iz app...",4,en,0.003112,0.070366,0.926522
525,Tut was es tun muss und dies tut es gut.,5,en,0.199186,0.759826,0.040988
